# CVA Calculation for an Interest Rate Swap

- Calculate the credit valuation adjustment to the price of an interest rate swap using the credit spreads for Counterparty B.
- Plot MtM values (a good plot will show results from many simulations)
- Produce smoothed Expected Exposure prodile using the mean of the exposure distribution - distribution of Forward LIBORs at each time $T_i+1$.
- Produce Potential Future Exposure with the simulated $L_6M$ taken from the $97.5^{th}$ percentile.

The details for the IRS are as follows:

Recovery Rate = 40%

Tenor = 5Y

Payments Frequency = 6M

MtM Position = Floating Leg - Fixed Leg

Counterparty A = Sovereign UK (http://data.cnbc.com/quotes/GBCD5) -> 43.00

Counterparty B = Sovereign Germany (http://data.cnbc.com/quotes/DECD5) -> 20.245

Credit spread value as per CNBC = 22.755 basis points (0.2275%, 0.002275)

We will need to compute:

- Fwd LIBOR rates (via monte carlo)
- Discount Factors
- Exposure
- Expected Exposure

And once we have all of those parts we will be able to determine the CVA (Credit Valuation Adjustment) for the interest rate swap outlined above.

### Forward LIBOR

To provide the $L_6M$ structure, we will generate the Forward LIBOR using _One Factor Libor Market Model_, described in 'Advanced Quantative Finance', Alonso Peña. 

I have a particular interest in parallel and GPU based computing so I took it as an opportunity to rewrite the reference material in Python to aid integration with the CUDA GPU library provided by nVidia (https://developer.nvidia.com/cuda-toolkit) and also the Intel MKL libraries for optimized Math functions on intel processors. These are conveniently provided by the default install of Anaconda (http://www.continuum.io) and are utilitised under the thirty day free trial.


In [2]:
import numpy as np
import accelerate as acc
from app.gpuCheck import getGpuCount

from accelerate.cuda.rand import PRNG, QRNG

gpuEnabled = False # gpu acceleration is not available in jupyter..
debug = False # code here is pretty much commented so set to false here..

# print 'Checking for compatible GPU ...'
# if getGpuCount() < 1:
#     print 'no compatible GPU installed, will revert to using numpy libraries for RNG'
# else:
#     print 'nVidia GPU(s) found: ', getGpuCount()
#     print 'Enabling GPU for RNG'
#     gpuEnabled = True

The notebook itself runs the normal versions of this, if you are interested in running the GPU enabled versions. You will need an NVidia GFX card with compute capability of greater than 2.0 and have installed the latest cuda drivers. Once installed you can run the python files from disk. Cuda code is included but commented out in the notebook, it can cause kernel panics and is somewhat unstable when running in an iPython environment.

### Utility functions

Let's setup some utility functions for random number generation. (the CUDA code is included here but commented out since it will cause kernel panics in Jupyter)

### Calculate Credit Default Swap Spreads

In order to infer the correct lambda to use, we need to calculate the spread value for the CDS. 

So for the value of the default leg, we will use:

$PV(default) = \sum_{i=1}^{T_i}N . (1-R) . DF_i . PD(T_i,T_{i-1}) $

$PV(premium) = \sum_{i=1}^{T_i} \pi . N . \Delta t . DF_i . PD(T_i) $

Bootstrapping the hazard rates using the formula below:

$\lambda_k = \frac{-1}{\Delta t} ln ( \frac{P(T_{k-1}D(0,T_k)(1-R)+\sum{n=1}^{k-1}}{} ) $

#### Discount Factors
The following formula has been used to derive the discount factors:

$DF_i = exp(-S_iT_i)$

#### Default Probabilities
The following formula has been used to derive the default probablities:

$PD_i = exp(-\lambda T_{i-1}) -exp(-\lambda T_i)\quad \forall i = 1,2,3,4$

#### Forward Rates
next we need the forward rates and the discount factors.

$L_I = S_I$

$L_1 = \frac{S_iT_i - S_{i-1}T_{i-1}}{T_i-T_{i-1}}$

Forward rates can be derived from the spot rates in continous time.

To encapsulate this I have created the CDS class below:

In [1]:
from numpy import array, ones, zeros, linspace, exp, put, sum, log

"""
above are the imports form the NumPy packages www.numpy.org
they come as part of the Anaconda distribution from continuum.io
"""

class CreditDefaultSwap(object):
    def __init__(self, N=float, timesteps=int, discountFactors=list, lamda=float, seed=float):
        self.N = N  # notional
        self.__lamda = lamda
        self.__seed = seed
        self.recoveryRate = 0.4
        self.Dt = 0.25
        self.timesteps = linspace(0, 1, timesteps + 1)
        self.discountFactors = discountFactors
        self.pT = self.generateProbSurvival()
        self.premiumLegsSpread = self.calculatePremiumLegSpreads()
        self.__premiumLegSum = sum(self.premiumLegsSpread)
        self.pD = self.generateProbDefault()
        self.defaultLegsSpread = self.calculateDefaultLegSpreads()
        self.__defaultLegSum = sum(self.defaultLegsSpread)
        self.fairSpread = self.premiumLegsSpread / self.defaultLegsSpread

    @property
    def premiumLegSum(self):
        return self.__premiumLegSum

    @property
    def defaultLegSum(self):
        return self.__defaultLegSum

    @property
    def markToMarket(self):
        return self.__premiumLegSum - self.__defaultLegSum

    @property
    def lamda(self):
        return self.__lamda

    @property
    def seed(self):
        return self.__seed

    def generateProbSurvival(self):
        """
        using $\exp^{-\lambda*T_i}$
        :return: 
        """
        pt = ones(len(self.timesteps))
        for index, t in enumerate(self.timesteps):
            if t > 0:
                ps = exp(self.lamda * -1 * t)
                put(pt, index, ps)
        return pt

    def generateProbDefault(self):
        """
        using $P(T,0) = P_{i-1} - P_i$
        :return: 
        """
        pd = zeros(len(self.timesteps))
        for index, pt in enumerate(self.pT):
            if index > 0:
                pdi = self.pT[index - 1] - pt
                put(pd, index, pdi)

        return pd

    def calculatePremiumLegSpreads(self):
        """
        returns the list of the premium leg values
        :return: array
        """
        #          assume 1%
        spreads = zeros(len(self.timesteps))
        for index, (df, pt) in enumerate(zip(self.discountFactors, self.pT)):
            if index > 0:
                spread = self.N * self.Dt * self.seed * df * pt
                put(spreads, index, spread)
        return spreads

    def calculateDefaultLegSpreads(self):
        """
        returns the list of the default leg values
        :return: array
        """
        #          assume 1%
        spreads = zeros(len(self.timesteps))
        for index, (df, pd) in enumerate(zip(self.discountFactors, self.pD)):
            if index > 0:
                spread = self.N * (1 - self.recoveryRate) * df * pd
                put(spreads, index, spread)
        return spreads

df = [0, 0.9975, 0.9900, 0.9779, 0.9607]
seed = 0.01 # initial guess
payments = 4
notional = 1000000

# fixed hazard rate
lamda = 0.03

c = CreditDefaultSwap(N=notional, timesteps=payments, discountFactors=df, lamda=lamda, seed=seed)

beforeOptimisation = c.markToMarket

print beforeOptimisation

-7772.71007138


We need to now compute what is the value that should be used. So by using an optimising algorithm we can minimise the value of markToMarket to 0. This will then give us the value that we should use for the hazard rate $\lambda$.

In [2]:
def calibrateCDS(lamda=float, seed=0.01):
    global CreditDefaultSwap
    c = CreditDefaultSwap(N=notional, timesteps=payments, discountFactors=df, lamda=lamda, seed=seed)
    return c.markToMarket

from scipy.optimize import fsolve

calibratedLambda = fsolve(calibrateCDS, lamda)

print calibratedLambda

[ 0.01663204]


The optimisation routine is provided by parts of the SciPy Optimize library, so we will need to import this. it needs to be wrapped in function so that the optimiser is able to call it with different values.

In [5]:
c = CreditDefaultSwap(N=notional, timesteps=payments, discountFactors=df, lamda=calibratedLambda, seed=seed)

print c.markToMarket

-1.81898940355e-12


Which is pretty close to zero so we can go ahead and use this value for our lambda in our simulation. Of course that was using some test data to validate that our implementation is correct.

The real data will come from the BOE spot curve which we will used a starting point for our simulations.

We will need to generate a large number of random numbers, these are normally provided by the methods in the numpy.random package but there are also others such as sobol and halton which are 3rd party libraries. 

## Random Number Generation

In order to facilitate simplified random number generation, it seemed appropriate to setup a utilty class to do this. It has also given the opportunity to explore the ability to use the GPU for both Pseudo Random Number Generation and Quasi Random Number Generation.

Additional libraries:

ghalton - https://github.com/fmder/ghalton

sobol_seq - https://github.com/naught101/sobol_seq

conda (package manager for Anaconda) does not install these properly so it is best to just clone and install it in the normal python way:

```python
cd itemToInstall/
python setup.py build
python setup.py test #(sometimes this fails, when no tests have been written)
python setup.py install #(sometimes requires admin privs depending on way python has been installed)]
```

these items are included on the usb drives.

In [13]:
import ghalton
import sobol_seq
from accelerate.cuda.rand import PRNG, QRNG
from numpy import array, empty, random, square, log, sqrt


def getPseudoRandomNumbers_Uniform(length=int):
    """

    generates a an array of psuedo random numbers from uniform distribution using numpy

    :param length:
    :return:
    """
    return random.uniform(size=length)


def getPseudoRandomNumbers_Uniform_cuda(length=int):
    # type: (object) -> object
    """

    generates a an array of psuedo random numbers from uniform distribution using CUDA

    :rtype: ndarray
    :param length:
    :return:
    """
    prng = PRNG(rndtype=PRNG.XORWOW)
    rand = empty(length)
    prng.uniform(rand)

    return rand


def getPseudoRandomNumbers_Standard(shape=tuple):
    """

    generates a an array of psuedo random numbers from standard normal distribution using numpy

    :param length:
    :return:
    """
    return random.normal(size=shape)


def getPseudoRandomNumbers_Standard_cuda(shape=tuple):
    # type: (object) -> object
    """

    generates a an array of psuedo random numbers from standard normal distribution using CUDA

    :rtype: ndarray
    :param length:
    :return:
    """
    prng = PRNG(rndtype=PRNG.XORWOW)
    rand = empty(shape)
    prng.normal(rand, 0, 1)

    return rand


def getSOBOLseq_standard(shape=tuple):
    """
    generate a SOBOL sequence
    
    :param shape: tuple of row, column of the desired return matrix
    :return:
    """
    return sobol_seq.i4_sobol_generate_std_normal(shape)


# def getSOBOLseq_uniform(shape=tuple):
#     return sobol_seq.i4_uniform(shape)

def getSOBOLseq_cuda(length=int):
    """

    returns an nd array of supplied length containing a SOBOL sequence of int64

    only for use on systems with CUDA libraries installed

    :param length:
    :return ndarray:
    """
    qrng = QRNG(rndtype=QRNG.SOBOL64)
    rand = empty(shape=length, dtype=int)
    qrng.generate(rand)

    return rand


def getHaltonSeq(dimensions=int, length=int):
    """

    returns an array Halton sequence of quasi random number

    :param dimensions: number of dimensions of matrix (columsn)
    :param length: number of sequences returned (rows)
    :return:
    """
    sequencer = ghalton.Halton(dimensions)
    return sequencer.get(length)


def getBoxMullerSample(randSource=array):
    t = 1.0
    while t >= 1.0 or t == 0.0:
        randValues = random.choice(randSource, 2)
        x = randValues[0]
        y = randValues[1]
        t = square(x) + square(y)

    result = x * sqrt(-2 * log(t) / t)
    return result


## Monte Carlo Simulation

We will now use a Monte Carlo simulation to compute the expected < DL > and expected < PL >. With the expectation of the fair spread being:

$ E [ s ] = \frac{ \langle DL \rangle}{\langle PL \rangle} $

To bootstrap the LIBOR curve we will use the One Factor Libor Model [1]. Originally implemented in C++, this has been recoded in Python. It could be argued that this would imply a slight loss in performance from C++, with C++ being as low level as possible without becoming assembler. An optimisation for python here would be to use a Cython wrapper to get as close as possible to the original C. One of the the languages advantages here is not to have to manage memory and pointers manually. 


In [1]:
# taken from BOE spot curve data
initRates_BOE_6m = [0.423546874, 0.425980925, 0.45950348, 0.501875772, 0.551473011, 0.585741857, 0.626315731,
                    0.667316554, 0.709477279, 0.753122018]

# to simulate the L6M






next we need the forward rates and the discount factors.

$L_I = S_I$

$L1 = \frac{S_iT_i - S_{i-1}T_{i-1}}{T_i-T_{i-1}}$

Forward rates can be derived from the spot rates in continous time.


#### Discount Factors
$DF_i = exp(-S_iT_i)$

#### Default Probabilities
$PD_i = exp(-\lambda T_{i-1}) -exp(-\lambda T_i)\quad \forall i = 1,2,3,4$

# Conclusions

Credit Model pricing is particularly suited to computational methods since it can be a laborious process to get all the mechanics of the models in place. 

### References

[1] _'The One Factor Libor Market Model Using Monte Carlo Simulation: An Empirical Investigation'_, Pena, di Sabatino,Ligato,Ventura,Bertagna. Dec 2010

[2] _'Advanced Quantitative Finance with C++'_, Pena. 2014

[3] _'Python for Finance'_, Yves Hilpisch. 2014

[4] _'emscriptem'_, http://kripken.github.io/emscripten-site/. - C++ to JavaScript compiler.

[5] _'VisPy'_, http://cyrille.rossant.net/compiler-data-visualization/

[3] _'Title'_, Author, (Mon YYYY)
